In [1]:
import torch
from torch import nn
from torch.optim import AdamW, SGD
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import torch.nn.functional as F
from torch.utils.data import DataLoader
import gc
import numpy as np
import os
import itertools
import math

from sonar.inference_pipelines.text import EmbeddingToTextModelPipeline
from sentence_transformers import SentenceTransformer

import wandb
import traceback

from transformers.utils import logging
logging.set_verbosity_error()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
d_emb = 1024

pad = torch.from_numpy(np.load('/workspace/eloise/sentemb/data/pad.npy')).to(device)

cos = nn.CosineSimilarity(dim=-1)
cossim = nn.CosineEmbeddingLoss()
mse = nn.MSELoss()

def criterion(output, target):
    
    mask = ~(target == pad).all(dim=-1) #[batch, seq]
    target = target[mask]
    output = output[mask]
    
    cossim_ = cossim(output, target, torch.full((1,), 1).to(device))
    mse_ = mse(output, target)
    return [cossim_, mse_]

'''
def nan_clip_grad(model):
    for param in model.named_parameters():
        if param.grad is not None:
            if torch.isnan(param.grad).any():
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1)

    
def check_for_nans(model):
    for name, param in model.named_parameters():
        for attr_name, attr_value in [("weights", param.data), ("grad", param.grad)]:
            if param.grad is not None:
                if torch.isnan(attr_value).any():
                    print(f"NaN detected in {attr_name} of {name}")
                    print("Stopping training")
                    return True
    return False
    

def check_for_nans(model):
    for name, param in model.named_parameters():
        if param.grad is not None:
            if torch.isnan(param.grad).any():
                print(f"NaN detected in weights of {name}")
                print("Stopping training")
                #tune.report({'Cossim': 10, 'MSE': 10})
                wandb.log({'Cossim': 10, 'MSE': 10})
                return True
    return False
'''

'\ndef nan_clip_grad(model):\n    for param in model.named_parameters():\n        if param.grad is not None:\n            if torch.isnan(param.grad).any():\n                torch.nn.utils.clip_grad_norm_(model.parameters(), 1)\n\n    \ndef check_for_nans(model):\n    for name, param in model.named_parameters():\n        for attr_name, attr_value in [("weights", param.data), ("grad", param.grad)]:\n            if param.grad is not None:\n                if torch.isnan(attr_value).any():\n                    print(f"NaN detected in {attr_name} of {name}")\n                    print("Stopping training")\n                    return True\n    return False\n    \n\ndef check_for_nans(model):\n    for name, param in model.named_parameters():\n        if param.grad is not None:\n            if torch.isnan(param.grad).any():\n                print(f"NaN detected in weights of {name}")\n                print("Stopping training")\n                #tune.report({\'Cossim\': 10, \'MSE\': 10})\n     

In [ ]:
def validation(epoch, cpt, model, valloader, val_list):
    cossim_loss = 0; mse_loss = 0
    model.eval()
    with torch.no_grad():
        for src in valloader:
            src = src.to(device)
            outputs = model.predict_next_sentence(src[:, :-1])
            [cossim_, mse_] = criterion(outputs, src[:, 1:])
            cossim_loss += cossim_.item(); mse_loss += mse_.item()
        cossim_loss = cossim_loss/len(valloader)
        mse_loss = mse_loss/len(valloader)
        val_list.append([cossim_loss, mse_loss])
        print('Epoch', epoch+1, 'Part', cpt, "Cossim", cossim_loss, "MSE", mse_loss)
        #tune.report({"Cossim": cossim_loss, "MSE": mse_loss})
        wandb.log({'Cossim': cossim_loss, 'MSE': mse_loss})
    model.train()
    return val_list, cpt+1


def autoregr_infer(model, prompts):
    seq = 20-1
    list_autoregr = []
    model.eval()
    with torch.no_grad():
        prompts = prompts.unsqueeze(0)
        for src in prompts:
            src = src.to(device)
            #autoregr = torch.cat((src, model(src)), dim=1)
            autoregr = torch.cat((src, model.predict_next_sentence(src)), dim=1)
            for i in range(seq-1):
                #outputs = model(autoregr)[:, -1].unsqueeze(1)
                outputs = model.predict_next_sentence(autoregr)[:, -1].unsqueeze(1)
                autoregr = torch.cat((autoregr, outputs), dim=1)
            list_autoregr.append(autoregr[:, 1:])
    return torch.cat(list_autoregr, dim=0)


def calculate_score(output, targets):
    seq = 20-1

    score_one_sum = 0
    score_sum_pad = torch.zeros(seq).to(device)
    pad_nbr_sum = torch.zeros(seq).to(device)

    for batch, target in enumerate(targets):
        out = output[batch][:len(target)].to(device)
        
        score = cos(out, target)

        score_one_sum += score.mean()
        score_sum_pad += F.pad(score, (0, seq - len(score)))
        pad_nbr_sum += F.pad(torch.ones(len(score)), (0, seq - len(score))).to(device)

    paragraphed_score = score_sum_pad/pad_nbr_sum
    paragraphed_score = [round(elem.item(), 2) for elem in paragraphed_score]
    final_score = score_one_sum.item()/len(targets)
    return final_score, paragraphed_score


def test(model, test_data, config):
    
    sonarprompt, sonaroutput, jasperoutput = test_data
    
    output_autoregr = autoregr_infer(model, sonarprompt)
    final_sonar, paragraphed_sonar = calculate_score(output_autoregr, sonaroutput)

    print('Sonar score:', final_sonar, paragraphed_sonar)
    wandb.log({'Final sonar': final_sonar, 'Paragraphed sonar': paragraphed_sonar})

    vec2text_model = EmbeddingToTextModelPipeline(decoder="text_sonar_basic_decoder", tokenizer="text_sonar_basic_decoder", device=device)

    text_autoregr = []
    for data in output_autoregr[:10]:
        text_autoregr.append(vec2text_model.predict(data, target_lang="eng_Latn", max_seq_len=64))
    
    torch.cuda.empty_cache(); del vec2text_model, sonarprompt, sonaroutput, output_autoregr; gc.collect()


    jasper = SentenceTransformer("infgrad/jasper_en_vision_language_v1",
        trust_remote_code=True,
        device=device,
        model_kwargs={"torch_dtype":  torch.bfloat16 if device == torch.device('cuda') else torch.float32},
    ); jasper.max_seq_length = 1024

    jasper_emb = []
    for data in text_autoregr:
        jasper_emb.append(torch.from_numpy(jasper.encode(data)).to(device))

    torch.cuda.empty_cache(); del jasper; gc.collect()
    
    
    final_jasper, paragraphed_jasper = calculate_score(jasper_emb, jasperoutput)
    
    torch.cuda.empty_cache(); del jasper_emb, jasperoutput; gc.collect()

    if final_sonar > 0.4 or final_jasper > 0.4:
        #torch.save(model.state_dict(), '/workspace/eloise/sentemb/Base_LCM_' + str(round(final_sonar, 2)) + '_' + str(round(final_jasper, 2)) + '_' + str(layers) + '-' + str(heads) + '-' + str(hidden_dim)+ '.pth')
        #torch.save([model.state_dict(), config], '/workspace/eloise/sentemb/Base_LCM_' + str(round(final_sonar, 2)) + '_' + str(round(final_jasper, 2)) + '.pth')
        torch.save([model.state_dict(), config], '/workspace/eloise/sentemb/Diff_LCM_' + str(round(final_sonar, 2)) + '_' + str(round(final_jasper, 2)) +'_'+str(config['model_dim'])+'_'+str(config['model_arch'][0])+'_'+str(config['model_arch'][1]) + '.pth')

    print('Jasper score:', final_jasper, paragraphed_jasper)
    wandb.log({'Final jasper': final_jasper, 'Paragraphed jasper': paragraphed_jasper})
    
    print('Final score:', (final_sonar+final_jasper)/2)
    wandb.log({'Final score': (final_sonar+final_jasper)/2})

In [3]:
from lcm.models.base_lcm.archs import toy_base_lcm

In [2]:
#from lcm.models.two_tower_diffusion_lcm.archs import toy_two_tower_diffusion_lcm
from lcm.models.two_tower_diffusion_lcm.archs import toy_lcm

In [3]:
from lcm.models.two_tower_diffusion_lcm.archs import toy_lcm
from lcm.models.two_tower_diffusion_lcm.builder import create_two_tower_diffusion_lcm_model

model = create_two_tower_diffusion_lcm_model(toy_lcm())

In [5]:
from lcm.models.base_lcm.archs import toy_base_lcm, base_lcm_1_6B, base_lcm_tuner, base_lcm_max
from lcm.models.base_lcm.builder import BaseLCModelBuilder

model = BaseLCModelBuilder(toy_base_lcm()).build_model('lolo')

In [3]:
from lcm.models.two_tower_diffusion_lcm.archs import toy_lcm
from lcm.models.two_tower_diffusion_lcm.builder import create_two_tower_diffusion_lcm_model

def objective(config, data):
    
    traindata, valloader, test_data = data
    
    #model = BaseLCModelBuilder(base_lcm_tuner(config), device=device).build_model(config.sonar_normalizer_name)
    #model = BaseLCModelBuilder(base_lcm_max(config), device=device).build_model(config.sonar_normalizer_name)
    #model = BaseLCModelBuilder(base_lcm_max(config), device=device).build_model()
    model = create_two_tower_diffusion_lcm_model(toy_lcm())
    model_save = model
    
    optimizer = AdamW(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)

    if config.scheduler:
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=config.T_0)
    else:
        scheduler = None

    val_list=[]
    cpt = 1
    loss_type=1 #COSSIM:0   MSE:1
    patience = 3

    checker = 6400/config.batch_size

    print('')
    
    for epoch in itertools.cycle(range(10)):
        if epoch!=0:
                torch.cuda.empty_cache(); del epoch_traindata, trainloader; gc.collect()
        if epoch%10==0:
            epoch_traindata = traindata
        else:
            epoch_traindata = torch.from_numpy(np.load('/workspace/eloise/sentemb/data/'+str(epoch)+'00k.npy'))
            
        trainloader = DataLoader(epoch_traindata, config.batch_size)
        
        model.train()

        for i, src in enumerate(trainloader):
            optimizer.zero_grad()
            src = src.to(device)
            outputs = model.predict_next_sentence(src[:, :-1])
            loss = criterion(outputs, src[:, 1:])[loss_type]
            loss.backward()
            
            if config.clip_grad:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            
            optimizer.step()
            
            if scheduler is not None:
                scheduler.step()
            
            if i!=0 and i%checker==0:
                val_list, cpt = validation(epoch, cpt, model, valloader, val_list)
                
                if math.isnan(val_list[-1][1]):
                    if len(val_list) > 1:
                        test(model_save, test_data, config)
                    else:
                        wandb.log({'Final score': -1})
                    break
                
                #if cpt==3:
                #    test(model_save, test_data, config)
                
                if val_list[0][1]> 0.8: wandb.log({'Cossim': val_list[-1][0], 'MSE': val_list[-1][1]}); test(model_save, test_data, config); break
                if cpt==5 and val_list[-1][1]> 0.01: wandb.log({'Cossim': val_list[-1][0], 'MSE': val_list[-1][1]}); test(model_save, test_data, config); break
                if cpt==30 and val_list[-1][1]> 5e-5: wandb.log({'Cossim': val_list[-1][0], 'MSE': val_list[-1][1]}); test(model_save, test_data, config); break
                
                #early stopping code:
                if len(val_list) >= 2:
                    loss_list = [x[loss_type] for x in val_list]
                    if loss_list[-1] < min(loss_list[:-1]):#it performs better, we save the model
                        model_save = model
                    elif (len(loss_list) - loss_list.index(min(loss_list))) > patience: #no better model in the last epochs
                        print('Best: Part', cpt-1-patience, 'Cossim', val_list[-1-patience][0], 'MSE', val_list[-1-patience][1])
                        wandb.log({'Cossim': val_list[-1-patience][0], 'MSE': val_list[-1-patience][1]})
                        torch.cuda.empty_cache(); del src, outputs, model, optimizer, valloader, trainloader, epoch_traindata, traindata, loss; gc.collect()
                        
                        #compute Final score
                        test(model_save, test_data, config)
                        break
        else:
            continue
        break

In [4]:
sweep_conf = {
    "method": "bayes",
    "metric": {"goal": "maximize", "name": "Final score"},
    "parameters": {
        "batch_size": {"values": [8, 16, 32, 64]},
        "scheduler": {"values": [True, False]},
        "T_0": {"min": 10, "max": 20000},
        "lr": {"distribution": "log_uniform_values", "min": 0.00001, "max": 0.1},
        "weight_decay": {"distribution": "log_uniform_values", "min": 0.00001, "max": 0.1},
        "clip_grad": {"values": [True, False]},
        
        #"model_dim": {"values": [512, 1024, 2048]},
        "model_dim": {"values": [2048]},
        #"num_attn_heads": {"values": [16]},
        
        #"sonar_normalizer_name": {"values": ["dummy_sonar_normalizer", "layernorm", None]},

        #"frontend_dropout_p": {"min": 0.0, "max": 0.5},
        #"frontend_pre_linear_init_fn": {"values": ['xavier', 'sonar', 'zero', 'trunc_normal', 'kaiming_uniform', 'none']},
        #"frontend_scale_embeddings": {"values": [True, False]},
        #"frontend_weight_normalization": {"values": [True, False]},

        #"lcm_final_dropout_p": {"min": 0.0, "max": 0.5},
        #"lcm_attention_dropout_p": {"min": 0.0, "max": 0.5},
        #"lcm_dropout_p": {"min": 0.0, "max": 0.5},
        #"lcm_ffn_inner_dim": {"values": [1, 2, 4]},
        "lcm_ffn_inner_dim": {"values": [2]},
        #"lcm_num_layers": {"values": [2, 8, 16, 24, 32]},
        #"lcm_num_layers": {"values": [2, 8, 14, 24, 32]},
        "lcm_num_layers": {"values": [18]},
        #"lcm_pos_embedding_style": {"values": ["rope", "sine", "learned", "none"]},
        #"lcm_use_swiglu": {"values": [True, False]},
        #"lcm_ffn_inner_activation_name": {"values": ["relu", "tanh", "elu", "leaky_relu", "prelu", "selu", "gelu", "silu", "softsign", "sigmoid", "hardsigmoid", None]},
        #"lcm_ffn_inner_activation_name": {"values": ["relu", "tanh", "elu", "leaky_relu", "selu", "gelu", "silu", "softsign", "sigmoid", "hardsigmoid", None]},
        #"lcm_layer_normalization_style": {"values": ["standard", "fp32", "rms", "unit"]},
        #"lcm_norm_order_style": {"values": ['pre', 'post', 'normformer']},
        #"lcm_final_norm_order_style": {"values": ['pre', 'post', 'normformer']},
        #"lcm_enable_qk_layernorm": {"values": [True, False]},
        #"lcm_mha_qkv_weight_normalization": {"values": [True, False]},
        #"lcm_mha_output_weight_normalization": {"values": [True, False]},
        #"lcm_mha_output_proj_bias": {"values": [True, False]},
        #"lcm_attention_output_init_fn": {"values": ['xavier', 'sonar', 'zero', 'trunc_normal', 'kaiming_uniform', 'none']},

        #"postnet_dropout_p": {"min": 0.0, "max": 0.5},
        #"postnet_linear_init_fn": {"values": ['xavier', 'sonar', 'zero', 'trunc_normal', 'kaiming_uniform', 'none']},
        #"postnet_weight_normalization": {"values": [True, False]},
        #"postnet_layer_normalization_style": {"values": ["standard", "fp32", "rms", "unit"]},
        #"postnet_activation_name": {"values": ["relu", "tanh", "elu", "leaky_relu", "prelu", "selu", "gelu", "silu", "softsign", "sigmoid", "hardsigmoid", None]},
    },
}

np_data = np.load('data/100k_1k_0.npz')
traindata = torch.from_numpy(np_data['train'])
valloader = DataLoader(torch.from_numpy(np_data['val']), batch_size=64)
test_data = torch.load('data/test_sonarprompt_sonaroutput_jasperoutput.pth')
torch.cuda.empty_cache(); del np_data; gc.collect()


#os.environ["WANDB_SILENT"] = "true"

def main():
    wandb.init()
    try:
        objective(wandb.config, [traindata, valloader, test_data])
    except Exception as e:
        print(e)
        traceback.print_exc()
        wandb.log({'Final score': -1})
    
sweep_id = wandb.sweep(sweep_conf, project="LCM")

wandb.agent(sweep_id, function=main)

/tmp/ipykernel_675054/466776071.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_data = torch.load('data/test_sonarprompt_sonaroutput_jasperoutput.pth')
wandb: Usin

Create sweep with ID: ue8adjip
Sweep URL: https://wandb.ai/seperability/LCM/sweeps/ue8adjip


wandb: Agent Starting Run: oftpht28 with config:
wandb: 	T_0: 2287
wandb: 	batch_size: 16
wandb: 	clip_grad: False
wandb: 	lcm_ffn_inner_dim: 2
wandb: 	lcm_num_layers: 18
wandb: 	lr: 0.0019910084515098784
wandb: 	model_dim: 2048
wandb: 	scheduler: False
wandb: 	weight_decay: 0.00011578422986358649
wandb: Currently logged in as: eloise_benito (seperability) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



TwoTowerDiffusionLCModel.predict_next_sentence() missing 1 required positional argument: 'context'


Traceback (most recent call last):
  File "/tmp/ipykernel_675054/466776071.py", line 64, in main
    objective(wandb.config, [traindata, valloader, test_data])
  File "/tmp/ipykernel_675054/1773245199.py", line 45, in objective
    outputs = model.predict_next_sentence(src[:, :-1])
  File "/workspace/eloise/large_concept_model/.venv/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
TypeError: TwoTowerDiffusionLCModel.predict_next_sentence() missing 1 required positional argument: 'context'


...


TwoTowerDiffusionLCModel.predict_next_sentence() missing 1 required positional argument: 'context'


Traceback (most recent call last):
  File "/tmp/ipykernel_675054/466776071.py", line 64, in main
    objective(wandb.config, [traindata, valloader, test_data])
  File "/tmp/ipykernel_675054/1773245199.py", line 45, in objective
    outputs = model.predict_next_sentence(src[:, :-1])
  File "/workspace/eloise/large_concept_model/.venv/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
TypeError: TwoTowerDiffusionLCModel.predict_next_sentence() missing 1 required positional argument: 'context'


Final score,▁
Final score,-1


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: x2pjr8hx with config:
wandb: 	T_0: 16953
wandb: 	batch_size: 64
wandb: 	clip_grad: False
wandb: 	lcm_ffn_inner_dim: 2
wandb: 	lcm_num_layers: 18
wandb: 	lr: 1.4380294269063304e-05
wandb: 	model_dim: 2048
wandb: 	scheduler: False
wandb: 	weight_decay: 1.4288913651147253e-05
wandb: Ctrl + C detected. Stopping sweep.


message_loop has been closed
Traceback (most recent call last):
  File "/workspace/eloise/large_concept_model/.venv/lib/python3.10/site-packages/wandb/sdk/interface/router_sock.py", line 27, in _read_message
    return self._sock_client.read_server_response(timeout=1)
  File "/workspace/eloise/large_concept_model/.venv/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 235, in read_server_response
    data = self._read_packet_bytes(timeout=timeout)
  File "/workspace/eloise/large_concept_model/.venv/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 220, in _read_packet_bytes
    raise SockClientClosedError
wandb.sdk.lib.sock_client.SockClientClosedError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/workspace/eloise/large_concept_model/.venv/lib/python3.10/site-packages/wandb/sdk/interface/router.py", line 56, in message_loop
    msg = self._read_message()
  File "/workspace/eloise/large_co